<a href="https://colab.research.google.com/github/MichaelHalaj/IT-202-001/blob/main/Logistic_Regression_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
#in read_csv, read only the first 1000 rows from each column to test it

In [2]:
from google.colab import files
files.upload()
#{'kaggle.json': b'{"username":"michaelhalaj","key":"bdb4c5734628864eae3507d48993d1f2"}'}

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"michaelhalaj","key":"bdb4c5734628864eae3507d48993d1f2"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download -c avazu-ctr-prediction

 99% 1.18G/1.19G [00:07<00:00, 157MB/s]
100% 1.19G/1.19G [00:07<00:00, 180MB/s]


In [7]:
! unzip avazu-ctr-prediction.zip 

Archive:  avazu-ctr-prediction.zip
  inflating: sampleSubmission.gz     
  inflating: test.gz                 
  inflating: train.gz                


In [17]:
import numpy as np
import pandas as pd
#YOU HAVE TO PREPROCESS THE DATA BEFORE READING IT CUZ ITS TOO MUCH INFO
#USE THE IDS TO FIND USEFUL DATA ON KAGGLE
# Logistic Regression https://www.youtube.com/watch?v=yIYKR4sgzI8
#https://asperbrothers.com/blog/logistic-regression-in-python/
""", usecols= ['id', 'click', 'hour', 'site_category', 'banner_pos', 'C1'],"""
rows = 100
""", usecols= ['id', 'click', 'hour', 'site_category', 'banner_pos', 'C1'], squeeze = True,"""
df=pd.read_csv('train.gz', nrows = rows)
df
#df.dropna(axis = 0)
#df.loc[:,'click']
#df.loc[:,['hour', 'banner_pos']]

#df['ones'] = np.ones((rows,), dtype=int)
#df.info()
#df_test = pd.read_csv('test.gz', nrows = 1000)
#df_z_scaled = df.copy()
#xs = np.array(df.loc[:,['banner_pos', 'ones']])
#ys = np.array(df.loc[:,'click'])
# apply normalization technique to Column 1
#column = 'banner_pos'
#df_z_scaled[column] = (df_z_scaled[column] - df_z_scaled[column].mean()) / df_z_scaled[column].std()
#df_z_scaled[column]


,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,10000371904215119486,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,10000640724480838376,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10015376300289320595,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15701,320,50,1722,0,35,100084,79
96,10015405794859644629,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15701,320,50,1722,0,35,100084,79
97,10015629448289660116,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15708,320,50,1722,0,35,-1,79
98,100156980486870304,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,-1,79


In [10]:
from pandas.compat.numpy.function import ndarray
import random
from typing import List

#There might be negative banner positions which cannot be entered into a log
Vector = List[float]

def logistic(x: float) -> float:
  return 1.0 / (1 + np.exp(-x))

def logistic_prime(x :float) -> float:
  y = logistic(x)
  return y * (1 - y)

def _negative_log_likelihood(x: Vector, y: float, beta: Vector) -> float:
  if y == 1:
    return -np.log(logistic(np.dot(x, beta)))
  else:
    return -np.log(1 - logistic(np.dot(x, beta)))

def negative_log_likelihood(xs: List[Vector], ys: List[float], beta: Vector) -> float:
  return sum(_negative_log_likelihood(x, y, beta) for x,y in zip(xs, ys))

def _negative_log_partial_j(x: Vector, y: float, beta: Vector, j: int) -> float:
  return -(y - logistic(np.dot(x, beta))) * x[j]

def _negative_log_gradient(x: Vector, y: float, beta: Vector) -> Vector:
  return [_negative_log_partial_j(x, y, beta, j) for j in range(len(beta))]

def negative_log_gradient(xs: List[Vector], ys: List[float], beta: Vector) -> Vector:
  return np.sum([_negative_log_gradient(x, y, beta) for x, y in zip(xs, ys)])

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
  
  step = np.multiply(step_size, gradient)
  return np.add(v, step)

import tqdm
import random

random.seed(0)
x_train = xs
y_train = ys

#print(x_train)
learning_rate = 0.01

beta = [random.random() for _ in range(2)]
with tqdm.trange(100) as t:
  for epoch in range(100):
    gradient = negative_log_gradient(x_train, y_train, beta)
    beta = gradient_step(beta, gradient, -learning_rate)
    loss = negative_log_likelihood(x_train, y_train, beta)
    t.set_description(f"loss: {loss:.3f} beta: {beta}")
  
print(beta)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
loss: inf beta: [-447.16864038 -447.25510783]:   0%|          | 0/100 [00:05<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
from matplotlib import pyplot as plt
import math
import random

#xs = df.drop(['click'], axis = 1)
#xs = df_z_scaled.loc[:,['banner_pos', 'hour']]
#ys = df.loc[:,'click']
xs = df[['banner_pos', 'hour']]
ys = df['click']
#xs = np.vectorize(xs)
xs = xs.astype(float)
ys = ys.astype(int)

print(xs)
print(ys)





def logistic(x: float) :
  return 1.0 / 1 (1 + math.exp(-x))

def logistic_prime(x: float):
  y = logistic(x)
  return y * (1 - y)

def _negative_log_likelihood(x, y, beta):
  if y == 1:
    return -math.log(logistic(np.dot(x, beta)))
  else:
    return -math.log(1 - logistic(np.dot(x, beta)))

def negative_log_likelihood(xs, ys, beta):
  return sum(_negative_log_likelihood(x, y, beta) for x, y in zip(xs, ys))

def _negative_log_partial_j(x, y, beta, j):
  return -(y - logistic(np.dot(x, beta))) * x[j]

def _negative_log_gradient(x, y, beta):
  return [_negative_log_partial_j(x, y, beta, j) for j in range(len(beta))]

def negative_log_gradient(xs, ys, beta):
  return np.sum([_negative_log_gradient(x, y, beta) for x, y in zip(xs, ys)])

def gradient_step(v, gradient, step_size):
  
  step = np.multiply(step_size, gradient)
  return np.add(v, step)

"""Grus Page 99"""
learning_rate = 0.01

beta = [random.random() for _ in range(3)]
print(beta)

for epoch in range(500):
  gradient = negative_log_gradient(xs, ys, beta)
  beta = gradient_step(beta, gradient, -learning_rate)
  loss = negative_log_likelihood(xs, ys, beta)


In [ ]:
#Use the code from the textbook to solvet this assignment as this will 